In [ ]:
%pip install transformers

In [ ]:
MODEL_NAME = "sentence-transformers/bert-base-nli-mean-tokens"

# Read data from the dictinoary excel file
sentences = ["no money can be lost",
             "this is perfectly secure",
             "there can be no safer place"]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

In [ ]:
tokens = {'input_ids': [], 'attention_mask': []}

In [ ]:
for sentence in sentences:
    new_tokens = tokenizer.encode_plus(sentence, max_length=128, truncation=True, padding='max_length', 
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

In [ ]:
#Tokenizing, stacking all the input_ids and attention_masks into one tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [ ]:
outputs = model(**tokens)

In [ ]:
embeddings = outputs.last_hidden_state

In [ ]:
attention = tokens['attention_mask']
mask = attention.unsqueeze(-1).expand(embeddings.shape).float()
mask_embeddings = embeddings * mask

In [ ]:
summed = torch.sum(mask_embeddings, 1)

In [ ]:
counts = torch.clamp(mask.sum(1), min=1e-9)

In [ ]:
mean_pooled = summed / counts

In [ ]:
mean_pooled = mean_pooled.detach().numpy()

np.save('prescriptiveStereotypes/safety.npy', mean_pooled)